# Fast Batch Multitask Lattice GP

In [1]:
import fastgp
import torch
import numpy as np

In [2]:
torch.set_default_dtype(torch.float64)

## True Function

In [3]:
def f(l,x):
    weights = 2**torch.arange(1,l+2)
    return torch.vstack([
        (torch.sin(weights*np.pi*x)/weights).sum(1),
        (torch.cos(weights*np.pi*x)/weights).sum(1),
    ])
num_tasks = 4
d = 1 # dimension
rng = torch.Generator().manual_seed(17)
x = torch.rand((2**7,d),generator=rng) # random testing locations
y = torch.cat([f(l,x)[:,None,:] for l in range(num_tasks)],1) # true values at random testing locations
z = torch.rand((2**8,d),generator=rng) # other random locations at which to evaluate covariance
print("x.shape = %s"%str(tuple(x.shape)))
print("y.shape = %s"%str(tuple(y.shape)))
print("z.shape = %s"%str(tuple(z.shape)))

x.shape = (128, 1)
y.shape = (2, 4, 128)
z.shape = (256, 1)


## Construct Fast GP

In [ ]:
fgp = fastgp.FastGPLattice(d,seed_for_seq=7,num_tasks=num_tasks,shape_batch=[2,])
x_next = fgp.get_x_next(n=2**torch.arange(4,4+num_tasks))
y_next = [f(l,x_next[l]) for l in range(num_tasks)]
fgp.add_y_next(y_next)
for i in range(len(x_next)):
    print("i = %d"%i)
    print("\tx_next[%d].shape = %s"%(i,str(tuple(x_next[i].shape))))
    print("\ty_next[%d].shape = %s"%(i,str(tuple(y_next[i].shape))))

i = 0
	x_next[0].shape = (16, 1)
	y_next[0].shape = (2, 16)
i = 1
	x_next[1].shape = (32, 1)
	y_next[1].shape = (2, 32)
i = 2
	x_next[2].shape = (64, 1)
	y_next[2].shape = (2, 64)
i = 3
	x_next[3].shape = (128, 1)
	y_next[3].shape = (2, 128)


In [5]:
pmean = fgp.post_mean(x)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-pmean,dim=-1)/torch.linalg.norm(y,dim=-1)))

pmean.shape = (2, 4, 128)
l2 relative error:
tensor([[2.3902e-02, 5.6744e-04, 4.1042e-05, 8.4138e-06],
        [2.3065e-02, 7.6313e-04, 4.6725e-05, 9.5641e-06]])


In [6]:
data = fgp.fit()
list(data.keys())

     iter of 5.0e+03 | NMLL       | norm term  | logdet term
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            0.00e+00 | -1.86e+03  | 1.86e+01   | -2.76e+03 
            5.00e+00 | -2.92e+03  | 2.23e+02   | -4.02e+03 
            1.00e+01 | -3.08e+03  | 5.74e+02   | -4.53e+03 
            1.50e+01 | -3.12e+03  | 4.48e+02   | -4.45e+03 
            2.00e+01 | -3.14e+03  | 4.73e+02   | -4.50e+03 
            2.50e+01 | -3.15e+03  | 4.85e+02   | -4.52e+03 
            3.00e+01 | -3.15e+03  | 4.91e+02   | -4.52e+03 
            3.50e+01 | -3.15e+03  | 4.79e+02   | -4.51e+03 
            4.00e+01 | -3.15e+03  | 4.76e+02   | -4.51e+03 
            4.50e+01 | -3.15e+03  | 4.72e+02   | -4.51e+03 
            5.00e+01 | -3.15e+03  | 4.81e+02   | -4.52e+03 
            5.50e+01 | -3.15e+03  | 4.70e+02   | -4.51e+03 
            6.00e+01 | -3.15e+03  | 4.71e+02   | -4.51e+03 
            6.50e+01 | -3.15e+03  | 4.74e+02   | -4.51e+03 
            7.00e+01 | -3.15e+03  | 4.

['mll_hist', 'scale_hist', 'lengthscales_hist', 'task_kernel_hist']

In [7]:
pmean,pvar,q,ci_low,ci_high = fgp.post_ci(x,confidence=0.99)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("pvar.shape = %s"%str(tuple(pvar.shape)))
print("q = %.2f"%q)
print("ci_low.shape = %s"%str(tuple(ci_low.shape)))
print("ci_high.shape = %s"%str(tuple(ci_high.shape)))
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-pmean,dim=-1)/torch.linalg.norm(y,dim=-1)))
pcov = fgp.post_cov(x,x)
print("pcov.shape = %s"%str(tuple(pcov.shape)))
_range0,_rangen1 = torch.arange(pcov.size(0)),torch.arange(pcov.size(-1))
assert torch.allclose(pcov[_range0,_range0][:,_rangen1,_rangen1],pvar) and (pvar>=0).all()
pcov2 = fgp.post_cov(x,z)
print("pcov2.shape = %s"%str(tuple(pcov2.shape)))

pmean.shape = (2, 4, 128)
pvar.shape = (4, 128)
q = 2.58
ci_low.shape = (2, 4, 128)
ci_high.shape = (2, 4, 128)
l2 relative error:
tensor([[3.1859e-04, 4.2987e-05, 2.4242e-05, 7.9237e-06],
        [2.9426e-04, 5.1505e-05, 2.3703e-05, 8.3381e-06]])
pcov.shape = (4, 4, 128, 128)
pcov2.shape = (4, 4, 128, 256)


In [8]:
pcmean,pcvar,q,cci_low,cci_high = fgp.post_cubature_ci(confidence=0.99)
print("pcmean:\n%s"%str(pcmean))
print("\npcvar:\n%s"%str(pcvar))
print("\ncci_low:\n%s"%str(cci_low))
print("\ncci_high:\n%s"%str(cci_high))

pcmean:
tensor([[-2.4317e-18,  1.3818e-17, -3.7545e-17, -7.3243e-18],
        [ 4.2735e-18,  1.2888e-17,  3.9807e-17, -2.9305e-17]])

pcvar:
tensor([7.2855e-09, 1.0397e-08, 3.9861e-09, 7.0173e-10])

cci_low:
tensor([[-2.1986e-04, -2.6264e-04, -1.6263e-04, -6.8234e-05],
        [-2.1986e-04, -2.6264e-04, -1.6263e-04, -6.8234e-05]])

cci_high:
tensor([[2.1986e-04, 2.6264e-04, 1.6263e-04, 6.8234e-05],
        [2.1986e-04, 2.6264e-04, 1.6263e-04, 6.8234e-05]])


## Project and Increase Sample Size

In [9]:
n_new = fgp.n*2**torch.arange(num_tasks-1,-1,-1)
pcov_future = fgp.post_cov(x,z,n=n_new)
pvar_future = fgp.post_var(x,n=n_new)
pcvar_future = fgp.post_cubature_var(n=n_new)

In [10]:
x_next = fgp.get_x_next(n_new)
y_next = [f(l,x_next[l]) for l in range(num_tasks)]
for _y in y_next:
    print(_y.shape)
fgp.add_y_next(y_next)
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-fgp.post_mean(x),dim=-1)/torch.linalg.norm(y,dim=-1)))
assert torch.allclose(fgp.post_cov(x,z),pcov_future)
assert torch.allclose(fgp.post_var(x),pvar_future)
assert torch.allclose(fgp.post_cubature_var(),pcvar_future)

torch.Size([2, 112])
torch.Size([2, 96])
torch.Size([2, 64])
torch.Size([2, 0])
l2 relative error:
tensor([[1.2902e-07, 5.1031e-07, 2.0854e-06, 5.6811e-06],
        [1.4225e-07, 5.5493e-07, 2.1187e-06, 6.1776e-06]])


In [11]:
data = fgp.fit(verbose=False,store_mll_hist=False,store_scale_hist=False,store_lengthscales_hist=False,store_noise_hist=False)
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-fgp.post_mean(x),dim=-1)/torch.linalg.norm(y,dim=-1)))

l2 relative error:
tensor([[1.0450e-07, 3.3484e-07, 1.1859e-06, 5.3415e-06],
        [1.1310e-07, 3.5550e-07, 1.2836e-06, 5.8452e-06]])


In [12]:
n_new = fgp.n*2**torch.arange(num_tasks)
pcov_new = fgp.post_cov(x,z,n=n_new)
pvar_new = fgp.post_var(x,n=n_new)
pcvar_new = fgp.post_cubature_var(n=n_new)
x_next = fgp.get_x_next(n_new)
y_next = [f(l,x_next[l]) for l in range(num_tasks)]
fgp.add_y_next(y_next)
assert torch.allclose(fgp.post_cov(x,z),pcov_new)
assert torch.allclose(fgp.post_var(x),pvar_new)
assert torch.allclose(fgp.post_cubature_var(),pcvar_new)